In [ ]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

In [ ]:
# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [ ]:
import nltk.corpus
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
vectorizer = CountVectorizer(ngram_range=(1,2),token_pattern=r'\b\w+\b',min_df=0)
tfTransformer = TfidfTransformer(use_idf = True)

In [ ]:
import string
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(string.punctuation)
stopwords.remove('not')
lemmatizer = WordNetLemmatizer()


In [ ]:
def cleanText(summary):
    summary = summary.replace('*','').replace('-',' ').replace('/',' ').replace("'",' ')
    tokens_summary = [str.lower().strip(string.punctuation) for str in summary.split() if str not in stopwords]
    lemma_summary = [lemmatizer.lemmatize(token) for token in tokens_summary if len(token) > 0]
    #for word in lemma_summary:
    #    wordList.append(word)
    return(' '.join(word for word in lemma_summary))
    

In [ ]:
words = []
classes = []
documents = []

# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        documents.append((pattern, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

In [ ]:
X = vectorizer.fit_transform([pattern for pattern,tag in documents])
Y = tfTransformer.fit_transform(X)

In [ ]:
output = []
output_empty = [0] * len(classes)
for doc in documents:
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    output.append(output_row)
len(output)

In [ ]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(vectorizer.get_feature_names())])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(classes), activation='softmax')
net = tflearn.regression(net)



In [ ]:
labels = np.reshape(classes, (-1, 1))

In [ ]:
labels

In [ ]:
# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)

model.fit(Y.toarray(), labels, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

In [ ]:
test_X = vectorizer.transform(["I'd like to rent a moped"])
test_Y = tfTransformer.transform(test_X)
ft = vectorizer.get_feature_names() 


result = list(map(lambda row:dict(zip(ft,row)),test_Y.toarray()))
#print(list(result[0].values()))
result_array = model.predict([list(result[0].values())]).tolist()[0]
result_array

In [ ]:

classes[result_array.index(max(result_array))]

In [ ]:
max(result_array)

In [1]:
from sklearn import svm
svmClassifier = svm.SVC(kernel='linear', C = 1.0,probability=True)


In [2]:
svmClassifier

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)